In [1]:
# Observacao: conflito com o nome `d`
# `d` ja existe como instancia Down(). Se voce quiser usar `d` como indice,
# guarde um alias antes de sobrescrever:
#
# from pylyra import *
# dn = d
# a, b, c, d = st.index('a b c d')
# # use dn(...) quando precisar da antiga instancia `d`


In [2]:
import sympy as sp
import pylyra as pl

# Simbolos gregos: https://en.wikipedia.org/wiki/Mathematical_operators_and_symbols_in_Unicode

# 𝛼	𝛽	𝛾	𝛿 	𝜀	𝜁	𝜂	𝜃	𝜄	𝜅	𝜆	𝜇	𝜈	𝜉	𝜊	𝜋	𝜌	𝜍	𝜎	𝜏
#	𝜐	𝜑	𝜒	𝜓	𝜔	𝜕	𝜖	𝜗	𝜘	𝜙	𝜚	𝜛

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

f1 = sp.Function('f1')
f2 = sp.Function('f2')

g = sp.diag(sp.exp(f1(r)), -sp.exp(f2(r)), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

# -- SpaceTime ----------
st = pl.SpaceTime(dim=4, coords=x, metric=g)

# -- Escala de Lyra ----------
print("Definindo escala")
phi = st.set_scale(sp.Function('phi')(r))

# -- Tensor de Levi Civita ----------
print("Definindo LeviCivita")
eps = st.tensor.from_function(
    lambda mu, nu, rho, sig: sp.sqrt(-st.detg) * sp.LeviCivita(mu, nu, rho, sig),
    signature=(pl.D, pl.D, pl.D, pl.D),
    name="eps",
)

# -- Non-Metricity ----------
print("Definindo Non-Metricity")
M = st.tensor.from_function(
    lambda a, b, c: 0,
    signature=(pl.u, pl.D, pl.D),
    name="M",
)
st.set_nonmetricity(M)

# -- Torsion ----------
print("Definindo Torsion")

def torsion_axial(mu):
    if mu == 0:
        return sp.Function('s_t')(r)
    if mu == 1:
        return sp.Function('s_r')(r)
    return 0

s = st.tensor.from_function(torsion_axial, signature=(pl.U,), name="s")


def torsion_trace(mu):
    if mu == 0:
        return sp.Function('v_t')(r)
    if mu == 1:
        return sp.Function('v_r')(r)
    return 0

v = st.tensor.from_function(torsion_trace, signature=(pl.D,), name="v")


def torsion_component(m, n, r_):
    return (
        sum(eps(pl.D, pl.D, pl.D, pl.D).comp[m, n, r_, sig] * s(pl.U).comp[sig] for sig in range(dim))
        + sp.Rational(1, 3) * (
            st.g[m, n] * v(pl.D).comp[r_]
            - st.g[m, r_] * v(pl.D).comp[n]
        )
    )

tau = st.set_torsion(st.tensor.from_function(torsion_component, signature=(pl.D, pl.D, pl.D), name="tau"))

st.update()


Definindo metrica
Definindo escala
Definindo LeviCivita
Definindo Non-Metricity
Definindo Torsion


In [3]:
a,b,c,d = st.index("a b c d")

eps[-a,-b,-c,-d](0,1,2,3)

r**2*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))

In [4]:
𝜇,𝜈,𝜌,𝜎 = st.index("mu nu rho sigma")

c = eps[-𝜇,-𝜈,-𝜌,-𝜎] * s[+𝜎] # o resultado disso deve ser: T2  shape=(4, 4, 4)  sig=___. Ta certo

# eu gostaria que o o _repr_html_ de IndexedTensor fosse igual aos arrays. Dessa forma, se e rodar

c[-𝜇,-𝜈,-𝜌]

# O resultado deixe de ser <pylyra.IndexedTensor at 0x75eb8a66b960> e passe a ser matrizes de matrizes renderizadas em MathJax; assim como é no sympy

[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, -r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, -r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0]], [[0, 0, 0, -r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, 0, r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, 0, 0], [r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), -r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0, 0]], [[0, 0, r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0], [0, 0, -r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0], [-r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0, 0], [0, 0, 0, 0]]]

In [7]:
Tem como fazer algumas alterações. Gostaria que se eu rodasse:
c[-𝜇,-𝜈,-𝜌](0,0)
aparecesse a primeira matriz 4x4.

E se eu rodar

c[-𝜇,-𝜈,-𝜌](0)

aparecesse uma matriz de matrizes 4x4x4



0

In [4]:
import sympy as sp

# Símbolos
x, y, z = sp.symbols("x y z")

# Um array 3D (2x2x3) com expressões algébricas/simbólicas
A = sp.MutableDenseNDimArray([
    [  # bloco i=0
        [x + y,      x*y,        x**2 + y**2],
        [sp.sin(x),  sp.exp(y),   (x - y)/z   ],
    ],
    [  # bloco i=1
        [x - y,      (x + 1)*(y + 2),  sp.sqrt(x**2 + y**2)],
        [sp.cos(x),  x**3 - y**3,      sp.log(z) + x*y     ],
    ],
])

A


[[[x + y, x*y, x**2 + y**2], [sin(x), exp(y), (x - y)/z]], [[x - y, (x + 1)*(y + 2), sqrt(x**2 + y**2)], [cos(x), x**3 - y**3, x*y + log(z)]]]

In [37]:
c[-𝜇,-𝜈,-𝜌]

In [ ]:
def torsion_component(m, n, r_):
    return (


        
        sum(eps(pl.D, pl.D, pl.D, pl.D).comp[m, n, r_, sig] * s(pl.U).comp[sig] for sig in range(dim))
        + sp.Rational(1, 3) * (
            st.g[m, n] * v(pl.D).comp[r_]
            - st.g[m, r_] * v(pl.D).comp[n]
        )
    )

tau = st.set_torsion(st.tensor.from_function(torsion_component, signature=(pl.D, pl.D, pl.D), name="tau"))

st.update()

In [ ]:
# Acesso a componentes apos indexacao

x, y = sp.symbols('x y')
st = SpaceTime(2, (x, y))

R = st.tensor.generic('R', (U, D, D, D))
a, b, c, d = st.index('a b c d')

# indexacao Einstein e acesso direto aos componentes
R[+a, -b, -c, -d](0, 1, 0, 1)
# forma explicita
R[+a, -b, -c, -d].get(0, 1, 0, 1)


In [3]:
R[+a, -b, -c, -d](0,1,0,1)

AttributeError: 'IndexedTensor' object has no attribute 'comp'

In [3]:
def fmt(expr):
    return sp.expand(sp.simplify(expr))

In [2]:
st.update()

In [4]:
fmt(st.ricci(d,d).comp[0,0])

s_r(r)**2*exp(f1(r))*exp(f2(r))/2 + v_r(r)**2*exp(f1(r))*exp(-f2(r))/9 + v_r(r)*exp(f1(r))*exp(-f2(r))*Derivative(f2(r), r)/(6*phi(r)) - exp(f1(r))*exp(-f2(r))*Derivative(v_r(r), r)/(3*phi(r)) + v_r(r)*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)/(3*phi(r)**2) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)**2/(4*phi(r)**2) + exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), (r, 2))/(2*phi(r)**2) + exp(f1(r))*exp(-f2(r))*Derivative(f2(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) - exp(f1(r))*exp(-f2(r))*Derivative(phi(r), (r, 2))/phi(r)**3 + 2*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)**2/phi(r)**4 - 2*v_r(r)*exp(f1(r))*exp(-f2(r))/(3*r*phi(r)) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)/(r*phi(r)**2) - 2*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)/(r*phi(r)**3)

In [7]:
fmt(st.ricci(u, u).comp[1,1])

s_t(r)**2*exp(f1(r))*exp(-f2(r))/2 + v_t(r)**2*exp(-f1(r))*exp(-f2(r))/9 + v_r(r)*exp(-2*f2(r))*Derivative(f1(r), r)/(6*phi(r)) + exp(-2*f2(r))*Derivative(f1(r), r)**2/(4*phi(r)**2) - exp(-2*f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) + exp(-2*f2(r))*Derivative(f1(r), (r, 2))/(2*phi(r)**2) + exp(-2*f2(r))*Derivative(f1(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) + 2*v_r(r)*exp(-2*f2(r))/(3*r*phi(r)) - exp(-2*f2(r))*Derivative(f2(r), r)/(r*phi(r)**2) + 2*exp(-2*f2(r))*Derivative(phi(r), r)/(r*phi(r)**3)

In [15]:
a = st.index("a")

fmt(st.contract(st.ricci.idx((a, None), (None, a))).expr)

3*s_r(r)**2*exp(f2(r))/2 - 3*s_t(r)**2*exp(f1(r))/2 + v_r(r)**2*exp(-f2(r))/3 - v_t(r)**2*exp(-f1(r))/3 - v_r(r)*exp(-f2(r))*Derivative(f1(r), r)/(2*phi(r)) + v_r(r)*exp(-f2(r))*Derivative(f2(r), r)/(2*phi(r)) - exp(-f2(r))*Derivative(v_r(r), r)/phi(r) + v_r(r)*exp(-f2(r))*Derivative(phi(r), r)/phi(r)**2 - exp(-f2(r))*Derivative(f1(r), r)**2/(2*phi(r)**2) + exp(-f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(2*phi(r)**2) - exp(-f2(r))*Derivative(f1(r), (r, 2))/phi(r)**2 - 3*exp(-f2(r))*Derivative(f1(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) + 3*exp(-f2(r))*Derivative(f2(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) - 3*exp(-f2(r))*Derivative(phi(r), (r, 2))/phi(r)**3 + 6*exp(-f2(r))*Derivative(phi(r), r)**2/phi(r)**4 - 2*v_r(r)*exp(-f2(r))/(r*phi(r)) - 2*exp(-f2(r))*Derivative(f1(r), r)/(r*phi(r)**2) + 2*exp(-f2(r))*Derivative(f2(r), r)/(r*phi(r)**2) - 6*exp(-f2(r))*Derivative(phi(r), r)/(r*phi(r)**3) + 2/(r**2*phi(r)**2) - 2*exp(-f2(r))/(r**2*phi(r)**2)

In [17]:
Rmn = st.riemann
Rcc = st.ricci
R = st.scalar_curvature

Agora, eu gostaria de mudar a sintaxe de:

Rmn = st.riemann

```python
Rmn.idx((a, None, None, None), (None, a, b, c))
```

para

```python
Rmn.up(a,None,None,None).down(None,a,b,c)
```

Além disso, faça esses dois métodos completarem a direito os indices nao fornecidos. Por exemplo:


```python
Rmn.up(a).down(None,a,b,c)
```

Nesse caso acima ficaria:

$
R^{a}_{\;abc}
$



In [2]:
from pylyra import *
# supondo que spaco

# componente: dT[a,b,k] = ∇_k T^a_b
dT.comp[0, 1, 2]


r**2*T_21(t, r, theta, varphi)*v_t(r)*exp(-f1(r))/3 + r**2*T_31(t, r, theta, varphi)*s_r(r)*sqrt(exp(f1(r) + f2(r)))*exp(-f1(r))*Abs(sin(theta))/2 - T_02(t, r, theta, varphi)*v_r(r)/3 - T_02(t, r, theta, varphi)*Derivative(phi(r), r)/phi(r)**2 + T_03(t, r, theta, varphi)*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))/(2*sin(theta)**2) + Derivative(T_01(t, r, theta, varphi), theta) - T_02(t, r, theta, varphi)/(r*phi(r))

In [8]:
st.gamma[0,0,0]

ValueError: Conexao nao definida.

# Sandbox

In [2]:
from pylyra import *

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

lamb = sp.Function('lambda')
nu = sp.Function('nu')

g = sp.diag(sp.exp(nu(r)), -sp.exp(lamb(r)), -r**2, -r**2*sp.sin(th)**2)

# -- SpaceTime ----------
st = SpaceTime(dim=4, coords=x, metric=g)

for i in range(3):
    display(
        sp.Eq(sp.expand(st.einstein(d, d).comp[i, i]), 0)
    )



Definindo metrica


Eq(-exp(-lambda(r))*exp(nu(r))*Derivative(lambda(r), r)/r - exp(nu(r))/r**2 + exp(-lambda(r))*exp(nu(r))/r**2, 0)

Eq(-Derivative(nu(r), r)/r + exp(lambda(r))/r**2 - 1/r**2, 0)

Eq(r**2*exp(-lambda(r))*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*Derivative(nu(r), r)/2, 0)

In [10]:
for i in range(3):
    display(
        sp.Eq(sp.expand(Ricc(d, d).comp[i, i] - sp.Rational(1,2) * st.g(d, d).comp[i, i] * R), 0)
    )



Eq(-exp(-lambda(r))*exp(nu(r))*Derivative(lambda(r), r)/r - exp(nu(r))/r**2 + exp(-lambda(r))*exp(nu(r))/r**2, 0)

Eq(-Derivative(nu(r), r)/r + exp(lambda(r))/r**2 - 1/r**2, 0)

Eq(r**2*exp(-lambda(r))*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*Derivative(nu(r), r)/2, 0)

In [4]:
R = st.contract(Ricc.idx((a, _), (_, a)))

In [1]:
from pylyra import *
import sympy as sp

# -------------------------
# Coords / functions
# -------------------------
t, r, th, vph = sp.symbols("t r theta varphi", real=True)
x = (t, r, th, vph)
dim = 4

a = sp.Function("a")   # scale factor a(t)
k = sp.symbols("k", real=True)  # curvature constant (often -1,0,+1)

# -------------------------
# FLRW metric: diag(+,-,-,-)
# -------------------------
g = sp.diag(
    1,
    -a(t)**2 / (1 - k*r**2),
    -a(t)**2 * r**2,
    -a(t)**2 * r**2 * sp.sin(th)**2
)

st = SpaceTime(dim=4, coords=x, metric=g)



# -------------------------
# If pylyra already exposes curvature tensors, use them.
# (Names may vary depending on your library.)
# -------------------------
try:
    # Example-style (adjust if your API uses different names)
    Ricc = st.ricci            # rank-2 tensor
    R = st.scalar_curvature    # scalar
    Ein = st.einstein          # rank-2 tensor

    # show Einstein equations G_{μν} = 0 (vacuum) or G_{μν} = 8πT_{μν}
    for i in range(4):
        print(f"G[{i},{i}] =")
        display(sp.simplify(Ein(d, d).comp[i, i]))

except Exception as e:
    print("Não achei st.ricci / st.einstein nessa versão do pylyra.")
    print("Erro:", e)
    print("Use a versão 2 abaixo (curvatura via from_function).")


G[0,0] =


3*(-k - Derivative(a(t), t)**2)/a(t)**2

G[1,1] =


(-k - 2*a(t)*Derivative(a(t), (t, 2)) - Derivative(a(t), t)**2)/(k*r**2 - 1)

G[2,2] =


r**2*(k + 2*a(t)*Derivative(a(t), (t, 2)) + Derivative(a(t), t)**2)

G[3,3] =


r**2*(k + 2*a(t)*Derivative(a(t), (t, 2)) + Derivative(a(t), t)**2)*sin(theta)**2

In [ ]:
print()


x = (t, r, th, vph) = sp.symbols('t r theta varphi', real=True)

st = SpaceTime(
    coords=x,
    metric=sp.diag(sp.exp(sp.Function('f1')(r)), -sp.exp(f2 = sp.Function('f2')(r)), -r**2, -r**2*sp.sin(th)**2),
)

# nesse ponto, phi=1, tau=0, M=0


# lembre da classe builtn Tensor. Ela deve receber uma métrica e conexão e escala no cconstrutor. Tensor deve ter informações sobre: nome (id unico), label (expressao para identificar o tensor, R, T, T_i) rank


# st deve ter um atributo interno .connection ou .gamma. Crie uma classe interna para abstrair isso


st.set_scale(sp.Function('phi')(r))

st.scale   #-> isso deve retornar uma instancia de Tensor de rank 0


st.set_torsion(..) # adicona a torção

st.set_non_metricity(...)


st.tensor.from_function(...)
st.tensor.generic(...)
st.tensor.generic(...)


# Algo parecido com isso seria legal


A = st.tensor.from_function( ..., signature=(d,d)) 
B = st.tensor.from_function( ..., signature=(d,d, d)) 


_,a,b,c = st.index("empty a b c")

st.contract(A.idx(up=(a,_),down=(.,a)), indexes=(a,)]  #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 0 

st.contract(A.idx(up=(a,b),down=(_,_)), B.idx(up=(_,_,_),down=(a,b,c))] #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 1


st.eval_contract("A^{a,}_{ ,a}")    #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 0 
st.eval_contract("A^{a,b} B_{a,b,c}") #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 1
            


Na verdade, queria melhorar as abstrações. Da forma como está agora:

- A métrica não é um tensor
- Escalares não são tensores
- Ter alguma forma de fazer contrações


Pensei em ter um elemento que abstraia o espaço-tempo. Manifold? Não sei

Dai, deveriamos ter métodos set_scale. Se não passar, usa-se phi = sp.Function('phi')(r)

um método set metric
set torsion
set metrica compatibility

e uma forma de fazer contração de indices


In [ ]:
M.contract(
    
)

In [1]:
from pylyra import *

t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

st = SpaceTime(
    dim=4,
    coords=x,
    metric=sp.diag(
        sp.exp(sp.Function('f1')(r)),
        -sp.exp(sp.Function('f2')(r)),
        -r**2,
        -r**2*sp.sin(th)**2
    ),
)

# defaults
st.scale      # Tensor rank 0 (phi = 1)
st.torsion    # tau = 0
st.nonmetricity  # M = 0

st.set_scale(sp.Function('phi')(r))

A = st.tensor.from_function(lambda i, j: sp.Function('A')(r), signature=(d, d), name="A")
B = st.tensor.from_function(lambda i, j, k: sp.Function('B')(r), signature=(d, d, d), name="B")

_, a, b, c = st.index("empty a b c")

# índices por posição (up/down por posição do rank)
res1 = st.contract(A.idx(up=(a, _), down=(_, a)))

# contração entre dois tensores
res2 = st.contract(
    A.idx(up=(a, b), down=(_, _)),
    B.idx(up=(_, _, _), down=(a, b, c))
)

# # parsing por string
# res3 = st.eval_contract("A^{a,}_{ ,a}")
# res4 = st.eval_contract("A^{a,b} B_{a,b,c}")


In [2]:
st.scale

AttributeError: 'ImmutableDenseNDimArray' object has no attribute '_repr_html_'

# Novos exemplos: indexacao segura e estrategias

Esta secao mostra a nova indexacao Einstein via `u()`/`d()` e o encadeamento de contracoes.


In [1]:
from pylyra import *

x, y = sp.symbols('x y')
st = SpaceTime(2, (x, y))

a, b, c = st.index('a b c')
T = st.generic('T', (u, d))
g = st.generic('g', (u, u))
V = st.generic('V', (d,))


In [2]:
# Contracao segura com indexacao Einstein
expr1 = T[u(a), d(b)] * g[u(b), u(c)]
expr1.signature


TypeError: '>=' not supported between instances of 'UpIndex' and 'Integer'

In [3]:
# Encadeamento de contracoes (A * B * C)
expr2 = T[u(a), d(b)] * g[u(b), u(c)] * V[d(c)]
expr2.signature


TypeError: '>=' not supported between instances of 'UpIndex' and 'Integer'

In [4]:
# Parse de string ainda funciona, agora com indices mistos em ordem horizontal
T3 = st.generic('T3', (u, d, u))
S3 = st.generic('S3', (d, u, d))
expr3 = st.eval_contract('T3^{a}{}_{b}{}^{c} S3_{a}{}^{b}{}_{c}')
expr3.signature


()

In [5]:
# Exemplo de estrategia de conexao fixa (Gamma = 0)
Gamma0 = sp.ImmutableDenseNDimArray([0] * (2**3), (2, 2, 2))
st2 = SpaceTime(2, (x, y), connection_strategy=FixedConnectionStrategy(Gamma0))
st2.update(include=['connection'])
st2.gamma.components


[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]